In [1]:
# Start writing code here...
print('hello-world')

hello-world


In [2]:
import numpy as np

In [3]:
list = [0, 1, 2]
nplist= np.asarray(list)
print(nplist)

[0 1 2]


In [4]:
URL='https://www.google.com/search?q=carbon+footprint'

import urllib
import requests
from bs4 import BeautifulSoup

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"

headers = {"user-agent" : MOBILE_USER_AGENT}
resp = requests.get(URL, headers=headers)

if resp.status_code == 200:
    soup = BeautifulSoup(resp.content, "html.parser")

# we could scrape the links with google docs :)
# https://www.youtube.com/watch?v=ObFKNDayudQ


<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="address=no" name="format-detection"/><meta content="origin" name="referrer"/><meta content="notranslate" name="google"/><link href="/images/branding/product/1x/gsa_android_144dp.png" rel="icon"/><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/><title>carbon footprint - Google Search</title><script nonce="HvC5/czcidgiDoiHqeituQ==">(function(){window.google={kEI:'oidbYb7mBtCy5NoPjpylkAs',kEXPI:'31',kBL:'rkvc'};google.sn='web';google.kHL='en';})();(function(){
var f=this||self;var h,k=[];function l(a){for(var b;a&&(!a.getAttribute||!(b=a.getAttribute("eid")));)a=a.parentNode;return b||h}function m(a){for(var b=null;a&&(!a.getAttribute||!(b=a.getAttribute("leid")));)a=a.p

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df8df42c-8503-4bb3-8c9d-497cf36bf82c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>